### 처리
1. 파라미터 설정
1. 매수 후보 종목 선별
1. 포트폴리오 조회 및 잔액갱신
1. 당일 스코아 계산
2. 매도 종목 검색 
4. 매수 종목 선정
1. (날짜 다음날 전환)
1. 매도처리
5. 매수처리


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import sqlite3
import csv
import random

%matplotlib notebook

In [2]:
# 전일영업일자 계산
def getBeforeWorkingDate(today):
    sql = "SELECT date FROM holiday_calendar WHERE date < {} AND holiday = 0".format(today)
    return pd.read_sql(sql, con, index_col=None).max()['date']

# 
def checkHoliday(day):
    sql = "SELECT holiday FROM holiday_calendar WHERE date = {}".format(day)
    return pd.read_sql(sql, con, index_col=None)['holiday'][0]

In [3]:
# *****************************
# 파라미터 설정
# *****************************
# 시작날짜
today = "20090312"
# 종료날짜
END_DAY = "20100106"
# 선별거래종목수
CANDIDATE_STOCK_CNT = 10
# 매수/매도 거래량
AMOUNT_BUY = 10
AMOUNT_SELL = 10
# 거래 주기 일자
CYCLE_DAY = 1

# directory
WORKING_DIR = "C:/Users/jseung21/Documents/CybosPlus"
# db file
dbDir = 'C:/Users/jseung21/Documents/CybosPlus/sqlite3'

In [4]:
# DBCP
con = sqlite3.connect(dbDir+"/MondayLab.db")
sqlite3.Connection
cursor = con.cursor()

### Loop 시작점

In [6]:
for gogo in range(1,3550):
    

    # 날짜 설정
    beforeDay = getBeforeWorkingDate(today)
    print("today:[{}], beforDay:[{}]".format(today,beforeDay))

    # *****************************
    # 매수 후보 종목 선별 TODO 지연
    # *****************************

    # kospi 200 종목 검색
    dfKospi200 = pd.read_sql("SELECT * FROM kospi200", con, index_col=None)

    # kospi200 종목중에서 매수 후보 주식 선별
    i = 0
    j = 0
    candidate = []
    for stockCode in dfKospi200['stockCode']:
        # 
        sqlTdoay = "SELECT stockCode FROM origin_trade WHERE 1=1 \
                    AND stockCode = '{}' \
                    AND date = {} \
                    AND days5MovAvr > days20MovAvr" \
                    .format(stockCode,today)
        sqlBeforeDay = "SELECT stockCode FROM origin_trade WHERE 1=1 \
                    AND stockCode = '{}' \
                    AND date = {} \
                    AND days5MovAvr <= days20MovAvr" \
                    .format(stockCode,beforeDay)

        result1 = pd.read_sql(sqlTdoay, con, index_col=None)

        if len(result1) != 0:
            result2 = pd.read_sql(sqlBeforeDay, con, index_col=None)

            if len(result2) != 0:
                j += 1
                candidate.append(result2['stockCode'][0])

        i += 1
    #     print("[{}]번째 진행중, [{}]데이터 보유".format(i,j))

    print("candidate : [{}]".format(len(candidate)))

    # 매수 후보 종목 csv 파일생성
    fileName = "{}/candidate_{}.csv".format(WORKING_DIR, today)
    with open(fileName, 'w') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(candidate)

    csvFile.close()

    # *****************************
    # 포트폴리오 조회 및 잔액갱신
    # *****************************
    sqlPortfolio = "SELECT seq, stockCode, amountBuy, priceBuy FROM Portfolio WHERE 1=1 \
                    AND trading = 1"
    portfolio = pd.read_sql(sqlPortfolio, con, index_col=None)

    print("portfolio 건수 : [{}] ".format(len(portfolio)))

    # 잔액 갱신
    for index, myStock in portfolio.iterrows():
        sqlFinalPrice = "select stockCode, finalPrice from origin_trade where 1=1 \
                        and stockCode = '{}' \
                        and date = {}" \
                        .format(myStock['stockCode'], today)
    #     print("sqlFinalPrice:[{}]".format(sqlFinalPrice))
        result = pd.read_sql(sqlFinalPrice, con, index_col=None)

        # update
        priceSell = result['finalPrice'][0]*AMOUNT_SELL
        balance = priceSell - myStock['priceBuy']
        sqlUpdate = "update portfolio set \
                    dateSell = {} \
                    , amountSell = {} \
                    , priceSell = {} \
                    , balance = {} \
                    where 1=1 \
                      and seq = {}" \
                    .format(today, AMOUNT_SELL, priceSell, balance, myStock['seq'])
    #     print("sqlUpdate:[{}]".format(sqlUpdate))
        cursor.execute(sqlUpdate)
    con.commit()

    # *****************************
    # 당일 스코아 계산
    # *****************************
    sqlScore = "select sum(priceBuy) as totalInvest, sum(priceSell) as toltalCurrVal from Portfolio"
    resultScore = pd.read_sql(sqlScore, con, index_col=None)
    if resultScore['totalInvest'][0] != None:
        totalInvest = resultScore['totalInvest'][0]
        toltalCurrVal = resultScore['toltalCurrVal'][0]
        score = round(float(toltalCurrVal) / float(totalInvest),5)

        sqlIstScore = "insert into Score values ({},{}, {},{})" \
                      .format(today, totalInvest, toltalCurrVal, score)
        print("sqlIstScore:[{}]".format(sqlIstScore))
        cursor.execute(sqlIstScore)
        con.commit()    
    
    # *****************************
    # 매도 종목 검색
    # *****************************
    stockSell = []
    if len(portfolio) != 0:
        for index, stcokSell in portfolio.iterrows():
            sqlStockSell = "SELECT stockCode FROM origin_trade WHERE 1=1 \
                            AND stockCode = '{}' \
                            AND date = {} \
                            AND days5MovAvr < days20MovAvr" \
                            .format(stcokSell['stockCode'], today)
    #         print("sqlStockSell:[{}]".format(sqlStockSell))
            sellStockCode = pd.read_sql(sqlStockSell, con, index_col=None)

            if len(sellStockCode) != 0:
                stockSell.append([stcokSell['seq'], sellStockCode['stockCode'][0], stcokSell['priceBuy']])

    print("stockSell 건수:[{}]".format(len(stockSell)))


    # *****************************
    # 매수 종목 선정
    # *****************************
    buyStockCount = CANDIDATE_STOCK_CNT - len(portfolio) + len(stockSell)
    print("buyStockCount : [{}]".format(buyStockCount))

    # list에서 random 추출
    buyStocks = 0
    if len(candidate) <= len(stockSell):
        buyStocks = len(candidate)
    else:
        buyStocks = random.sample(candidate, buyStockCount)


    # *****************************
    # (날짜 다음날 전환)
    # *****************************
    from datetime import datetime  
    from datetime import timedelta 

    temp = today
    for i in range(1,100):
        nextDate = datetime.strptime(str(today), "%Y%m%d") + timedelta(days=i)
        checkDay = nextDate.strftime('%Y%m%d')
        if checkHoliday(checkDay) == 0:
            break
    today = checkDay
    print("chage today [{}] to [{}]".format(temp, today))


    # *****************************
    # 매도 처리
    # *****************************
    if len(stockSell) != 0:
        for sellStockCode in stockSell:
            # 시초가 거래
            sqlSell = "select startPrice from origin_trade where 1=1 \
                        AND stockCode = '{}' \
                        AND date = {}" \
                        .format(sellStockCode[1], today)
    #         print("sqlSell:[{}]".format(sqlSell))
            startPrice = pd.read_sql(sqlSell, con, index_col=None)['startPrice'][0]

            # Trading 매도 내역 생성
            stockPrice = startPrice*AMOUNT_SELL
            sqlInstSellTrd = "insert into Trading values ({},{},'{}',{},{})" \
                            .format(today, 2, sellStockCode[1], AMOUNT_SELL, stockPrice)
    #         print("sqlInstSellTrd:[{}]".format(sqlInstSellTrd))
            cursor.execute(sqlInstSellTrd)

            # Portfolio 갱신
            balance = stockPrice - sellStockCode[2]
            seq = sellStockCode[0]
            sqlUpdate = "update portfolio set \
                        trading = 2 \
                        , dateSell = {} \
                        , amountSell = {} \
                        , priceSell = {} \
                        , balance = {} \
                        where 1=1 \
                          and seq = {}" \
                        .format(today, AMOUNT_SELL, stockPrice, balance, seq)
    #         print("sqlUpdate:[{}]".format(sqlUpdate))
            cursor.execute(sqlUpdate)

        con.commit()




    # *****************************
    # 매수처리
    # *****************************
    for stockBuyCode in buyStocks:
        # 시초가 거래
        sqlBuy = "select startPrice from origin_trade where 1=1 \
                  AND stockCode = '{}' \
                  AND date = {}" \
                  .format(stockBuyCode, today)
        startPrice = pd.read_sql(sqlBuy, con, index_col=None)['startPrice'][0]

    #     print("stockBuyCode:[{}], startPrice:[{}]".format(stockBuyCode, startPrice))

        # Trading 매수 내역 생성
        sqlInstBuyTrd = "insert into Trading values({},{},'{}',{},{})" \
                       .format(today, 1, stockBuyCode, AMOUNT_BUY, startPrice*AMOUNT_BUY)
        cursor.execute(sqlInstBuyTrd)

        # Portfolio 생성
        sqlSeq = "select max(seq) as maxSeq from Portfolio"
        seq = pd.read_sql(sqlSeq, con, index_col=None)['maxSeq'][0]
        if seq == None:
            seq = 0
        seq += 1
        trading = 1
        stockPrice = startPrice*AMOUNT_BUY

        sqlInsert = "insert into Portfolio values({},{},'{}',{},{},{},{},{},{},{})" \
                    .format(seq, trading, stockBuyCode, today, AMOUNT_BUY, stockPrice,0,0,0,0)
        cursor.execute(sqlInsert)

    con.commit()
    


today:[20090312], beforDay:[20090311]
candidate : [25]
portfolio 건수 : [0] 
stockSell 건수:[0]
buyStockCount : [10]
chage today [20090312] to [20090313]
today:[20090313], beforDay:[20090312]
candidate : [27]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20090313] to [20090316]
today:[20090316], beforDay:[20090313]
candidate : [19]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20090316] to [20090317]
today:[20090317], beforDay:[20090316]
candidate : [9]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20090317] to [20090318]
today:[20090318], beforDay:[20090317]
candidate : [4]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20090318] to [20090319]
today:[20090319], beforDay:[20090318]
candidate : [6]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20090319] to [20090320]
today:[20090320], beforDay:[20090319]
candidate : [2]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCou

ValueError: Sample larger than population or is negative